In [ ]:
# Pre- requisitos para executar o notebook
pip install pandas numpy matplotlib seaborn jupyter

In [3]:
# Análise de Qualidade de Dados - Critérios DMBOK
# Análise de 5 CSVs (um para cada ano) seguindo os critérios do DAMA-DMBOK

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [4]:
class DMBOKDataQualityAnalyzer:
    """
    Analisador de Qualidade de Dados baseado nos critérios do DAMA-DMBOK
    """
    
    def __init__(self):
        self.dataframes = {}
        self.quality_report = {}
        self.years = []
    
    def load_csvs(self, file_paths):
        """
        Carrega os CSVs para análise
        file_paths: lista de caminhos dos arquivos CSV
        """
        for i, path in enumerate(file_paths):
            year = 2019 + i  # Assumindo anos de 2019 a 2023
            try:
                df = pd.read_csv(path, encoding='utf-8')
                self.dataframes[year] = df
                self.years.append(year)
                print(f"✓ CSV {year} carregado: {len(df)} registros, {len(df.columns)} colunas")
            except Exception as e:
                print(f"✗ Erro ao carregar CSV {year}: {e}")
    
    def analyze_completeness(self):
        """
        Critério DMBOK: Completude (Completeness)
        Analisa dados ausentes, nulos e vazios
        """
        print("\n" + "="*60)
        print("ANÁLISE DE COMPLETUDE (COMPLETENESS)")
        print("="*60)
        
        completeness_data = []
        
        for year, df in self.dataframes.items():
            print(f"\n📊 Ano {year}:")
            
            # Análise por coluna
            missing_analysis = {}
            for col in df.columns:
                total_records = len(df)
                missing_count = df[col].isnull().sum()
                empty_strings = (df[col] == '').sum() if df[col].dtype == 'object' else 0
                total_missing = missing_count + empty_strings
                completeness_pct = ((total_records - total_missing) / total_records) * 100
                
                missing_analysis[col] = {
                    'total_records': total_records,
                    'missing_values': missing_count,
                    'empty_strings': empty_strings,
                    'total_incomplete': total_missing,
                    'completeness_percentage': completeness_pct
                }
                
                print(f"  {col}: {completeness_pct:.1f}% completo ({total_missing} valores incompletos)")
            
            # Completude geral do dataset
            overall_completeness = df.count().sum() / (len(df) * len(df.columns)) * 100
            print(f"\n  📈 Completude Geral: {overall_completeness:.1f}%")
            
            completeness_data.append({
                'year': year,
                'overall_completeness': overall_completeness,
                'column_analysis': missing_analysis
            })
        
        self.quality_report['completeness'] = completeness_data
    
    def analyze_consistency(self):
        """
        Critério DMBOK: Consistência (Consistency)
        Analisa padrões de dados entre anos e dentro de datasets
        """
        print("\n" + "="*60)
        print("ANÁLISE DE CONSISTÊNCIA (CONSISTENCY)")
        print("="*60)
        
        consistency_data = []
        
        # 1. Consistência estrutural (colunas)
        print("\n🔍 Consistência Estrutural:")
        all_columns = set()
        column_presence = {}
        
        for year, df in self.dataframes.items():
            cols = set(df.columns)
            all_columns.update(cols)
            column_presence[year] = cols
        
        # Verificar se todas as colunas estão presentes em todos os anos
        consistent_columns = all_columns.copy()
        for col in all_columns:
            years_present = [year for year, cols in column_presence.items() if col in cols]
            if len(years_present) != len(self.years):
                print(f"  ⚠️  Coluna '{col}' ausente em: {set(self.years) - set(years_present)}")
                consistent_columns.discard(col)
        
        structural_consistency = len(consistent_columns) / len(all_columns) * 100
        print(f"  📊 Consistência Estrutural: {structural_consistency:.1f}%")
        
        # 2. Consistência de tipos de dados
        print("\n🔍 Consistência de Tipos:")
        type_consistency = {}
        
        for col in consistent_columns:
            types_per_year = {}
            for year, df in self.dataframes.items():
                if col in df.columns:
                    types_per_year[year] = str(df[col].dtype)
            
            unique_types = set(types_per_year.values())
            is_consistent = len(unique_types) == 1
            type_consistency[col] = {
                'consistent': is_consistent,
                'types_by_year': types_per_year
            }
            
            if not is_consistent:
                print(f"  ⚠️  '{col}': Tipos inconsistentes - {types_per_year}")
        
        # 3. Consistência de categorias (para campos categóricos)
        print("\n🔍 Consistência de Categorias:")
        category_consistency = {}
        
        for col in consistent_columns:
            if any(df[col].dtype == 'object' for year, df in self.dataframes.items() if col in df.columns):
                categories_per_year = {}
                for year, df in self.dataframes.items():
                    if col in df.columns:
                        categories_per_year[year] = set(df[col].dropna().unique())
                
                # Verificar se as categorias são consistentes
                all_categories = set()
                for cats in categories_per_year.values():
                    all_categories.update(cats)
                
                consistent_categories = all_categories.copy()
                for cat in all_categories:
                    years_with_cat = [year for year, cats in categories_per_year.items() if cat in cats]
                    if len(years_with_cat) < len(self.years) * 0.8:  # Presente em pelo menos 80% dos anos
                        consistent_categories.discard(cat)
                
                category_consistency[col] = {
                    'total_categories': len(all_categories),
                    'consistent_categories': len(consistent_categories),
                    'consistency_percentage': len(consistent_categories) / len(all_categories) * 100 if all_categories else 100
                }
                
                print(f"  {col}: {category_consistency[col]['consistency_percentage']:.1f}% das categorias consistentes")
        
        consistency_data.append({
            'structural_consistency': structural_consistency,
            'type_consistency': type_consistency,
            'category_consistency': category_consistency
        })
        
        self.quality_report['consistency'] = consistency_data
    
    def analyze_accuracy(self):
        """
        Critério DMBOK: Precisão/Acurácia (Accuracy)
        Analisa valores válidos baseados nas regras de negócio
        """
        print("\n" + "="*60)
        print("ANÁLISE DE PRECISÃO/ACURÁCIA (ACCURACY)")
        print("="*60)
        
        accuracy_data = []
        
        for year, df in self.dataframes.items():
            print(f"\n📊 Ano {year}:")
            year_accuracy = {}
            
            for col in df.columns:
                col_accuracy = {'valid_values': 0, 'invalid_values': 0, 'accuracy_percentage': 0}
                
                # Análise específica por tipo de coluna
                if 'TIPO' in col.upper() or 'TP_' in col:
                    # Campos categóricos - verificar se seguem padrão numérico esperado
                    valid_values = df[col].dropna()
                    if valid_values.dtype in ['int64', 'float64']:
                        # Assumindo que códigos válidos são números positivos
                        valid_count = (valid_values > 0).sum()
                        invalid_count = len(valid_values) - valid_count
                    else:
                        # Para strings, verificar se não estão vazias
                        valid_count = (valid_values != '').sum()
                        invalid_count = (valid_values == '').sum()
                    
                    col_accuracy = {
                        'valid_values': valid_count,
                        'invalid_values': invalid_count,
                        'accuracy_percentage': (valid_count / len(valid_values)) * 100 if len(valid_values) > 0 else 0
                    }
                
                elif 'TAMANHO' in col.upper():
                    # Campo tamanho - deve ser numérico positivo
                    valid_values = df[col].dropna()
                    if len(valid_values) > 0:
                        try:
                            numeric_values = pd.to_numeric(valid_values, errors='coerce')
                            valid_count = (numeric_values > 0).sum()
                            invalid_count = len(valid_values) - valid_count
                            col_accuracy = {
                                'valid_values': valid_count,
                                'invalid_values': invalid_count,
                                'accuracy_percentage': (valid_count / len(valid_values)) * 100
                            }
                        except:
                            col_accuracy['accuracy_percentage'] = 0
                
                else:
                    # Para outros campos, verificar apenas se não são nulos/vazios
                    valid_values = df[col].dropna()
                    if df[col].dtype == 'object':
                        valid_count = (valid_values != '').sum()
                        invalid_count = (valid_values == '').sum()
                    else:
                        valid_count = len(valid_values)
                        invalid_count = 0
                    
                    col_accuracy = {
                        'valid_values': valid_count,
                        'invalid_values': invalid_count,
                        'accuracy_percentage': (valid_count / len(valid_values)) * 100 if len(valid_values) > 0 else 0
                    }
                
                year_accuracy[col] = col_accuracy
                print(f"  {col}: {col_accuracy['accuracy_percentage']:.1f}% preciso")
            
            # Precisão geral do ano
            overall_accuracy = np.mean([acc['accuracy_percentage'] for acc in year_accuracy.values()])
            print(f"\n  📈 Precisão Geral: {overall_accuracy:.1f}%")
            
            accuracy_data.append({
                'year': year,
                'overall_accuracy': overall_accuracy,
                'column_accuracy': year_accuracy
            })
        
        self.quality_report['accuracy'] = accuracy_data
    
    def analyze_validity(self):
        """
        Critério DMBOK: Validade (Validity)
        Verifica se os dados seguem formatos e regras esperados
        """
        print("\n" + "="*60)
        print("ANÁLISE DE VALIDADE (VALIDITY)")
        print("="*60)
        
        validity_data = []
        
        for year, df in self.dataframes.items():
            print(f"\n📊 Ano {year}:")
            year_validity = {}
            
            for col in df.columns:
                validity_issues = []
                valid_count = 0
                total_count = len(df[col].dropna())
                
                if total_count == 0:
                    year_validity[col] = {'validity_percentage': 0, 'issues': ['Coluna vazia']}
                    continue
                
                # Verificações específicas por tipo de campo
                if 'NOME' in col.upper() or 'DESCRICAO' in col.upper():
                    # Campos de texto - verificar comprimento mínimo
                    valid_values = df[col].dropna()
                    valid_count = (valid_values.str.len() >= 3).sum()
                    if valid_count < total_count:
                        validity_issues.append(f"{total_count - valid_count} valores com menos de 3 caracteres")
                
                elif 'TIPO' in col.upper() or 'TP_' in col:
                    # Campos categóricos - verificar se são números ou seguem padrão
                    valid_values = df[col].dropna()
                    if valid_values.dtype == 'object':
                        # Verificar se seguem padrão "N. Descrição"
                        pattern_matches = valid_values.str.match(r'^\d+\..*').sum()
                        valid_count = pattern_matches
                        if pattern_matches < total_count:
                            validity_issues.append(f"{total_count - pattern_matches} valores não seguem padrão 'N. Descrição'")
                    else:
                        # Para numéricos, verificar se são inteiros positivos
                        valid_count = ((valid_values > 0) & (valid_values % 1 == 0)).sum()
                        if valid_count < total_count:
                            validity_issues.append(f"{total_count - valid_count} valores não são inteiros positivos")
                
                elif 'TAMANHO' in col.upper():
                    # Campo tamanho - deve ser numérico
                    valid_values = df[col].dropna()
                    try:
                        numeric_values = pd.to_numeric(valid_values, errors='coerce')
                        valid_count = (~numeric_values.isna()).sum()
                        if valid_count < total_count:
                            validity_issues.append(f"{total_count - valid_count} valores não numéricos")
                    except:
                        validity_issues.append("Erro na conversão para numérico")
                        valid_count = 0
                
                elif 'CATEGORIA' in col.upper():
                    # Campo categoria - verificar formato estruturado
                    valid_values = df[col].dropna()
                    # Assumir que categorias válidas têm pelo menos uma numeração
                    pattern_matches = valid_values.str.contains(r'\d+\.', na=False).sum()
                    valid_count = pattern_matches
                    if pattern_matches < total_count:
                        validity_issues.append(f"{total_count - pattern_matches} categorias sem numeração")
                
                else:
                    # Para outros campos, assumir válidos se não estão vazios
                    valid_count = total_count
                
                validity_percentage = (valid_count / total_count) * 100 if total_count > 0 else 0
                year_validity[col] = {
                    'validity_percentage': validity_percentage,
                    'issues': validity_issues
                }
                
                print(f"  {col}: {validity_percentage:.1f}% válido")
                if validity_issues:
                    for issue in validity_issues:
                        print(f"    ⚠️  {issue}")
            
            # Validade geral do ano
            overall_validity = np.mean([val['validity_percentage'] for val in year_validity.values()])
            print(f"\n  📈 Validade Geral: {overall_validity:.1f}%")
            
            validity_data.append({
                'year': year,
                'overall_validity': overall_validity,
                'column_validity': year_validity
            })
        
        self.quality_report['validity'] = validity_data
    
    def analyze_timeliness(self):
        """
        Critério DMBOK: Atualidade (Timeliness)
        Analisa se os dados estão atualizados e relevantes
        """
        print("\n" + "="*60)
        print("ANÁLISE DE ATUALIDADE (TIMELINESS)")
        print("="*60)
        
        # Verificar se temos dados para anos consecutivos
        years_sorted = sorted(self.years)
        expected_years = list(range(min(years_sorted), max(years_sorted) + 1))
        missing_years = set(expected_years) - set(years_sorted)
        
        print(f"📅 Anos disponíveis: {years_sorted}")
        if missing_years:
            print(f"⚠️  Anos ausentes: {sorted(missing_years)}")
        
        # Calcular atualidade baseada na cobertura temporal
        temporal_coverage = len(years_sorted) / len(expected_years) * 100
        print(f"📊 Cobertura Temporal: {temporal_coverage:.1f}%")
        
        # Verificar se os dados mais recentes são do ano atual ou próximo
        current_year = 2025  # Ajustar conforme necessário
        most_recent_year = max(years_sorted)
        years_behind = current_year - most_recent_year
        
        timeliness_score = max(0, 100 - (years_behind * 20))  # Penalizar 20% por ano de atraso
        print(f"📈 Score de Atualidade: {timeliness_score:.1f}% (dados mais recentes: {most_recent_year})")
        
        self.quality_report['timeliness'] = {
            'temporal_coverage': temporal_coverage,
            'most_recent_year': most_recent_year,
            'years_behind': years_behind,
            'timeliness_score': timeliness_score
        }
    
    def generate_visualizations(self):
        """
        Gera visualizações dos resultados da análise
        """
        print("\n" + "="*60)
        print("GERANDO VISUALIZAÇÕES")
        print("="*60)
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle('Análise de Qualidade de Dados - Critérios DMBOK', fontsize=16, fontweight='bold')
        
        # 1. Completude por ano
        if 'completeness' in self.quality_report:
            years = [item['year'] for item in self.quality_report['completeness']]
            completeness = [item['overall_completeness'] for item in self.quality_report['completeness']]
            
            axes[0, 0].plot(years, completeness, marker='o', linewidth=2, markersize=8)
            axes[0, 0].set_title('Completude por Ano', fontweight='bold')
            axes[0, 0].set_xlabel('Ano')
            axes[0, 0].set_ylabel('Completude (%)')
            axes[0, 0].grid(True, alpha=0.3)
            axes[0, 0].set_ylim(0, 100)
        
        # 2. Precisão por ano
        if 'accuracy' in self.quality_report:
            years = [item['year'] for item in self.quality_report['accuracy']]
            accuracy = [item['overall_accuracy'] for item in self.quality_report['accuracy']]
            
            axes[0, 1].plot(years, accuracy, marker='s', linewidth=2, markersize=8, color='orange')
            axes[0, 1].set_title('Precisão por Ano', fontweight='bold')
            axes[0, 1].set_xlabel('Ano')
            axes[0, 1].set_ylabel('Precisão (%)')
            axes[0, 1].grid(True, alpha=0.3)
            axes[0, 1].set_ylim(0, 100)
        
        # 3. Validade por ano
        if 'validity' in self.quality_report:
            years = [item['year'] for item in self.quality_report['validity']]
            validity = [item['overall_validity'] for item in self.quality_report['validity']]
            
            axes[1, 0].plot(years, validity, marker='^', linewidth=2, markersize=8, color='green')
            axes[1, 0].set_title('Validade por Ano', fontweight='bold')
            axes[1, 0].set_xlabel('Ano')
            axes[1, 0].set_ylabel('Validade (%)')
            axes[1, 0].grid(True, alpha=0.3)
            axes[1, 0].set_ylim(0, 100)
        
        # 4. Resumo geral de qualidade
        if all(key in self.quality_report for key in ['completeness', 'accuracy', 'validity']):
            metrics = ['Completude', 'Precisão', 'Validade']
            scores = []
            
            # Calcular média de cada métrica
            completeness_avg = np.mean([item['overall_completeness'] for item in self.quality_report['completeness']])
            accuracy_avg = np.mean([item['overall_accuracy'] for item in self.quality_report['accuracy']])
            validity_avg = np.mean([item['overall_validity'] for item in self.quality_report['validity']])
            
            scores = [completeness_avg, accuracy_avg, validity_avg]
            
            bars = axes[1, 1].bar(metrics, scores, color=['skyblue', 'orange', 'lightgreen'])
            axes[1, 1].set_title('Scores Médios de Qualidade', fontweight='bold')
            axes[1, 1].set_ylabel('Score (%)')
            axes[1, 1].set_ylim(0, 100)
            
            # Adicionar valores nas barras
            for bar, score in zip(bars, scores):
                height = bar.get_height()
                axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 1,
                               f'{score:.1f}%', ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        plt.show()
    
    def generate_summary_report(self):
        """
        Gera um relatório resumo da análise
        """
        print("\n" + "="*80)
        print("RELATÓRIO RESUMO - ANÁLISE DE QUALIDADE DE DADOS (DMBOK)")
        print("="*80)
        
        print(f"\n📋 DATASETS ANALISADOS: {len(self.dataframes)} arquivos CSV")
        print(f"📅 PERÍODO: {min(self.years)} - {max(self.years)}")
        
        total_records = sum(len(df) for df in self.dataframes.values())
        total_columns = sum(len(df.columns) for df in self.dataframes.values())
        print(f"📊 VOLUME: {total_records:,} registros, {total_columns} colunas (total)")
        
        print(f"\n🎯 CRITÉRIOS DMBOK AVALIADOS:")
        
        # Completude
        if 'completeness' in self.quality_report:
            avg_completeness = np.mean([item['overall_completeness'] for item in self.quality_report['completeness']])
            print(f"   ✓ Completude (Completeness): {avg_completeness:.1f}%")
        
        # Precisão
        if 'accuracy' in self.quality_report:
            avg_accuracy = np.mean([item['overall_accuracy'] for item in self.quality_report['accuracy']])
            print(f"   ✓ Precisão (Accuracy): {avg_accuracy:.1f}%")
        
        # Consistência
        if 'consistency' in self.quality_report:
            structural_consistency = self.quality_report['consistency'][0]['structural_consistency']
            print(f"   ✓ Consistência (Consistency): {structural_consistency:.1f}%")
        
        # Validade
        if 'validity' in self.quality_report:
            avg_validity = np.mean([item['overall_validity'] for item in self.quality_report['validity']])
            print(f"   ✓ Validade (Validity): {avg_validity:.1f}%")
        
        # Atualidade
        if 'timeliness' in self.quality_report:
            timeliness_score = self.quality_report['timeliness']['timeliness_score']
            print(f"   ✓ Atualidade (Timeliness): {timeliness_score:.1f}%")
        
        # Score geral de qualidade
        if all(key in self.quality_report for key in ['completeness', 'accuracy', 'validity']):
            overall_quality = np.mean([avg_completeness, avg_accuracy, avg_validity])
            print(f"\n🏆 SCORE GERAL DE QUALIDADE: {overall_quality:.1f}%")
            
            if overall_quality >= 90:
                print("   🟢 EXCELENTE - Dados de alta qualidade")
            elif overall_quality >= 75:
                print("   🟡 BOM - Dados de qualidade satisfatória com pontos de melhoria")
            elif overall_quality >= 60:
                print("   🟠 REGULAR - Dados precisam de melhorias significativas")
            else:
                print("   🔴 CRÍTICO - Dados requerem ação imediata")
        
        print(f"\n📋 RECOMENDAÇÕES:")
        print(f"   • Implementar processos de validação de dados na origem")
        print(f"   • Estabelecer monitoramento contínuo da qualidade")
        print(f"   • Criar regras de negócio para campos categóricos")
        print(f"   • Padronizar formatos e estruturas entre anos")
        print(f"   • Implementar governança de dados conforme DMBOK")


In [7]:
def main():
    """
    Função principal para executar a análise
    """
    print("🚀 INICIANDO ANÁLISE DE QUALIDADE DE DADOS - CRITÉRIOS DMBOK")
    print("="*60)
    
    # Inicializar analisador
    analyzer = DMBOKDataQualityAnalyzer()
    
    # ALTERE AQUI OS CAMINHOS DOS SEUS ARQUIVOS CSV
    csv_files = [
        'MICRODADOS_CADASTRO_CURSOS_2019.csv', 
        'MICRODADOS_CADASTRO_CURSOS_2020.csv', 
        'MICRODADOS_CADASTRO_CURSOS_2021.csv', 
        'MICRODADOS_CADASTRO_CURSOS_2022.csv', 
        'MICRODADOS_CADASTRO_CURSOS_2023.csv'  
    ]
    
    # Carregar CSVs
    print("📂 Carregando arquivos CSV...")
    analyzer.load_csvs(csv_files)
    
    if not analyzer.dataframes:
        print("❌ Nenhum CSV foi carregado. Verifique os caminhos dos arquivos.")
        return
    
    # Executar análises dos critérios DMBOK
    analyzer.analyze_completeness()    # Completude
    analyzer.analyze_consistency()     # Consistência  
    analyzer.analyze_accuracy()        # Precisão/Acurácia
    analyzer.analyze_validity()        # Validade
    analyzer.analyze_timeliness()      # Atualidade
    
    # Gerar visualizações
    analyzer.generate_visualizations()
    
    # Gerar relatório resumo
    analyzer.generate_summary_report()
    
    print(f"\n✅ ANÁLISE CONCLUÍDA!")
    return analyzer

# Executar análise
if __name__ == "__main__":
    # Para executar, descomente a linha abaixo e ajuste os caminhos dos arquivos
    # analyzer = main()
    
    # Exemplo de como carregar dados de exemplo (descomente para testar)
    print("\n" + "="*60)
    print("EXECUÇÃO COM DADOS GERADOS EM 2019,2020,2021,2022 E 2023")
    print("="*60)
    
    # Criar dados de exemplo baseados na estrutura fornecida
    example_data = {
        'Nome da Variável': [
            'TP_ORGANIZACAO_ACADEMICA',
            'TP_CATEGORIA_ADMINISTRATIVA', 
            'IN_CAPITAL',
            'QT_DOCENTE_TOTAL',
            'QT_CURSO_TOTAL'
        ],
        'Descrição da Variável': [
            'Tipo de Organização Acadêmica da IES',
            'Tipo de Categoria Administrativa da IES',
            'Indicador se IES está na Capital',
            'Quantidade Total de Docentes',
            'Quantidade Total de Cursos'
        ],
        'Tipo': ['Num', 'Num', 'Num', 'Num', 'Num'],
        'Tamanho': [1, 1, 1, 6, 4],
        'Categoria': [
            '1. Universidade 2. Centro Universitário 3. Faculdade 4. Instituto Federal de Educação, Ciência e Tecnologia 5. Centro Federal de Educação Tecnológica',
            '1. Pública Federal 2. Pública Estadual 3. Pública Municipal 4. Privada com fins lucrativos 5. Privada sem fins lucrativos',
            '1. Sim 0. Não',
            'Quantidade',
            'Quantidade'
        ]
    }
    
    # Criar DataFrames de exemplo
    analyzer_example = DMBOKDataQualityAnalyzer()
    for year in [2019, 2020, 2021, 2022, 2023]:
        # Simular pequenas variações nos dados
        df_example = pd.DataFrame(example_data.copy())
        
        # Adicionar alguns valores ausentes e inconsistências para demonstrar
        if year == 2020:
            df_example.loc[1, 'Tipo'] = None  # Valor ausente
        if year == 2022:
            df_example.loc[2, 'Tamanho'] = 'erro'  # Valor inválido
        
        analyzer_example.dataframes[year] = df_example
        analyzer_example.years.append(year)

    # Executar análises no exemplo
    analyzer_example.analyze_completeness()
    analyzer_example.analyze_consistency()
    analyzer_example.analyze_accuracy()
    analyzer_example.analyze_validity()
    analyzer_example.analyze_timeliness()


EXECUÇÃO COM DADOS GERADOS EM 2019,2020,2021,2022 E 2023

ANÁLISE DE COMPLETUDE (COMPLETENESS)

📊 Ano 2019:
  Nome da Variável: 100.0% completo (0 valores incompletos)
  Descrição da Variável: 100.0% completo (0 valores incompletos)
  Tipo: 100.0% completo (0 valores incompletos)
  Tamanho: 100.0% completo (0 valores incompletos)
  Categoria: 100.0% completo (0 valores incompletos)

  📈 Completude Geral: 100.0%

📊 Ano 2020:
  Nome da Variável: 100.0% completo (0 valores incompletos)
  Descrição da Variável: 100.0% completo (0 valores incompletos)
  Tipo: 80.0% completo (1 valores incompletos)
  Tamanho: 100.0% completo (0 valores incompletos)
  Categoria: 100.0% completo (0 valores incompletos)

  📈 Completude Geral: 96.0%

📊 Ano 2021:
  Nome da Variável: 100.0% completo (0 valores incompletos)
  Descrição da Variável: 100.0% completo (0 valores incompletos)
  Tipo: 100.0% completo (0 valores incompletos)
  Tamanho: 100.0% completo (0 valores incompletos)
  Categoria: 100.0% completo 